# Know your (computer's) limits: Floating point arithmetic

*(Jupyter notebook by Simone Brugiapaglia)*

In this notebook we will illustrate some basic notions and experiments on floating point arithmetic. 

The main reference is Chapter 9 of the book

*Q. Kong, T. Siauw, and A. M. Bayen. Python Programming and Numerical Methods: A Guide for Engineers and Scientists. Academic Press, 2021. (https://pythonnumericalmethods.berkeley.edu/)*

In [1]:
# import all modules necessary to run this notebook
import sys
import numpy as np
import math

## Floating point numbers specifics (IEEE 754 standard)

A floating point number $n$ can be represented as
$$
n = (-1)^s \cdot 2^{c-1023} \cdot (1+f),
$$
where $s$ is the *sign indicator*, $c$ is the *characteristic* or *exponent* and $f$ is the *mantissa* or *fraction*. In the IEEE 754 standard for double precision arithmetic (see https://en.wikipedia.org/wiki/IEEE_754) a floating point number is stored using 64 bits, organized as follows: 
- 1 bit for the sign $s \in \{0,1\}$
- 11 bits for the characteristic $c \in \{0,1, \ldots, 2^{11}-1 = 2047\}$
- 52 bits for the mantissa (in binary representation) $f = 0.b_1b_2\ldots b_{52} \in [0, 1-2^{-52}]$ (where $b_j \in\{0,1\}$ are binary digits)

Note that $c = 0$ is reserved to represent the number $n=0$ and so-called *subnormal numbers* (https://en.wikipedia.org/wiki/Subnormal_number). In addition, $c = 2047$ is reserved for the special values `nan` (not a number) and `inf`.

In Python, we can obtain information about floating point numbers specifics using the `sys` module


In [2]:
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

### Smallest and largest floating point numbers

One could easily check what the largest and smallest double precision floating point numbers are.

Recalling that $c = 2047$ is reserved for special values, the largest flating point number is $2^{1023}\cdot(2-2^{-52})$, corresponding to $c = 2046$ and $f = 1-2^{-52}$. We can also compute it using the command `sys.float_info.max`.

In [3]:
c = 2046       # characteristic
f = 1-2**(-52) # mantissa
print(2**(c-1023)*(1+f))  # largest floating point number, computed using the definition

print(sys.float_info.max) # largest floating point number, computed using the sys module

1.7976931348623157e+308
1.7976931348623157e+308


Similary, one can compute the smallest floatin point number and show that it concides with `sys.float_info.min` (note that subnormal numbers, corresponding to the exponent 

In [4]:
c = 1
f = 0
print(2**(c-1023)*(1+f)) # largest floating point number, computed using the definition
sys.float_info.min       # largest floating point number, computed using the sys module

2.2250738585072014e-308


2.2250738585072014e-308

### Gap between floating point numbers

Note that real numbers have no gaps between them, since they contiuously fill up the real line. However, this is not the case for floating point numbers. In fact, there is always a nonzero gap between two consecutive floating point numbers. In practice, this means that, unlike the line of real numbers, the set of floating point numbers has a *finite* resolution. The gap between a given number and its closest neighbor can be computed using the Python command `np.spacing`, inluded in the `numpy` library. 

Let's compute the gap between `1` and its closest floating point number:

In [5]:
np.spacing(1) # compute the gap between 1 and its closest floating point number

2.220446049250313e-16

This means that if we add `2.220446049250313e-16` to `1`, we will get a number different from `1`. 

In [6]:
one_plus_small = 1 + np.spacing(1)
print(one_plus_small)      # add a small floating point number to 1, larger than its gap
print(one_plus_small == 1) # check if the result is equal to 1

1.0000000000000002
False


However, it we add, for example, one half of `2.220446049250313e-16` to `1`, the result will be inaccurate, since there is not enough space to represent that number in floating point arithmetic.

In [7]:
one_plus_smaller = 1 + np.spacing(1)/2
print(one_plus_smaller)      # add a small floating point number to 1, smaller than its gap
print(one_plus_smaller == 1) # check if the result is equal to 1

1.0
True


## Round-off errors

The previous examples where we add a very small number to `1` and get an inaccurate result is an example of *round-off error*. The fact that we perform operations in floating point arithmetic constantly introduces errors. 

The first kind of round-off error is the *representation error* committed by storing numbers that are not floating point numbers in memory. Examples include $e$, $\pi$, but also rational numbers such as $1/3$ or $3/7$. For example, $e$ has infinitely-many non periodic digits in its decimal representation. However, its floating point representation can only contain a finite number of digits.

In [8]:
print(math.e) # standard way to use e in Python

2.718281828459045


We could try to define a new variable contaning more digits of $e$ (see https://apod.nasa.gov/htmltest/gifcity/e.1mil), but the result will be truncated and only 64 bits of this number will be stored in memory.

In [9]:
# More digits of e (from https://apod.nasa.gov/htmltest/gifcity/e.1mil)
e_with_more_digits = 2.7182818284590452353602874713526624977572470936999595749669676277240766303535475945713821785251664274274663919320
print(e_with_more_digits)
print(e_with_more_digits == math.e) # Check that our representation of e with more digit is equal to the stardard one

2.718281828459045
True


Another type of round-off error is generated by floating point arithmetic. For example, $\cos(\pi/2)$ is not exactly equal to $0$ in because the computation of this function depends on floating point arithmetic operations.

In [10]:
math.cos(math.pi / 2)

6.123233995736766e-17

Let's consider another illustrative example where simple arithmetic operations can cause round-off errors. For example, we know that the expression $x + 1/x$ is always a number different from $x$, for any $x \neq 0$. However, the situation in floating point arithmetic is different and depends on the size of $x$... If $x$ is too large, then the sum $x + 1/x$ will yield a cancellation error, resulting in this number being equal to $x$!

In [11]:
x = 1e8
print(x == x + 1/x)

False


In [12]:
x = 1e9
print(x == x + 1/x)

True


### Two ways of computing the sample variance
The sample variance of a sample $x_1, \ldots, x_n$ is defined as 
$$
s^2 = \frac{1}{n-1}\sum_{i=1}^n (x_i - \bar{x})^2, \quad \text{where } \bar{x} = \frac1n \sum_{i=1}^n x_i,
$$
and $\bar{x}$ is the sample mean.

In [13]:
n = 1000
x = np.random.rand(n,1) # generate n random numbers uniformly distributed in [0,1]
sample_mean = np.sum(x) / n
sample_var = np.sum( (x - sample_mean)**2 ) / (n-1)

print('sample mean =', sample_mean)
print('sample variance (1st formula) =', sample_var)

sample mean = 0.4906384897802641
sample variance (1st formula) = 0.08591274059325583


An equivalent formula for the sample variance is 
$$
s^2 = \frac{1}{n-1} \left( \sum_{i=1}^n x_i^2 - n \bar{x}^2\right).
$$
(Try to prove that they are equivalent!)

In [14]:
sample_var_2 = (np.sum(x**2) - n * sample_mean**2 ) / (n-1)
print('sample variance (2nd formula) =',sample_var_2)

sample variance (2nd formula) = 0.08591274059325575


The results are essentially equivalent. However, if we add a very large number to the sample observations, only the first formula remains accurate. (Note that adding a constant number to the sample observations should only change its mean, not its variance).

In [15]:
x = x + 1e10
sample_mean = np.sum(x) / n
sample_var = np.sum( (x - sample_mean)**2 ) / (n-1)
sample_var_2 = (np.sum(x**2) - n * sample_mean**2 ) / (n-1)
sample_var_2

print('sample mean =', sample_mean)
print('sample variance (1st formula) =', sample_var)
print('sample variance (2nd formula) =', sample_var_2)

sample mean = 10000000000.490639
sample variance (1st formula) = 0.0859127453663172
sample variance (2nd formula) = 0.0


The second formula is subject to huge round-off error. Be careful, this could lead to false discoveries!

## Recommended problems

1. After reading Chapter 9 of 

    *Q. Kong, T. Siauw, and A. M. Bayen. Python Programming and Numerical Methods: A Guide for Engineers and Scientists. Academic Press, 2021. (https://pythonnumericalmethods.berkeley.edu/)*

    I invite you to work on the following problems: 1, 2, 6, 7.

2. Also, have fun trying to find examples where round-off errors leads to unexpected results!